# 1. Introdução

Usar uma heurística de detecção de pele para isolar regiões da imagem que provavelmente contêm faces humanas, sem usar nenhum classificador treinado.

A ideia principal se basea em utilizar o espaço de cores YCrCb.

O YCrCb (ou Y'CbCr) é um espaço de cor derivado do RGB, usado extensivamente em compressão de vídeo (como MPEG, JPEG) e transmissão de televisão digital. Ele separa:

* Y (luminância): brilho (intensidade da luz).
* Cr (chrominance-red): diferença entre o vermelho e o brilho.
* Cb (chrominance-blue): diferença entre o azul e o brilho.

Isso quer dizer:

Podemos ignorar o Y (que muda com sombra/luz) e focar apenas nos componentes Cr e Cb.

Assim, mesmo sob luz forte ou fraca, a cor relativa da pele permanece consistente nesse espaço.


A literatura mostra que os tons de pele humanos estão geralmente na faixa:

* Cr: 133 a 173
* Cb: 77 a 127

Independente de etnia, essas faixas conseguem capturar a maioria dos tons de pele. Isso não é tão consistente no HSV, onde a tonalidade (H) pode variar mais entre indivíduos e iluminação.

# 2. Estratégia

* Converter a imagem para o espaço de cor YCrCb.

* Aplicar um **limiar (threshold)** para extrair a máscara da pele.

* Aplicar morfologia para remover ruído.**(TAREFA)**

* Encontrar contornos ou regiões que pareçam uma face.

In [ ]:
! [ ! -d "/content/pessoas.jpg" ] && gdown -O /content/pessoas.jpg "19Q9M_vCwIxyUK4C45F2ZjRZcKUoBUugq"


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Tamanho da imagem (Cb x Cr)
size = 256
Y_fixed = 128

# Inicializa a imagem YCrCb
img_ycrcb = np.zeros((size, size, 3), dtype=np.uint8)

# Agora: i = Cr, j = Cb
for i in range(size):       # Cr
    for j in range(size):   # Cb
        img_ycrcb[i, j] = [Y_fixed, i, j]  # [Y, Cr, Cb] com Cr=linha, Cb=coluna

# Converte para BGR
img_bgr = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2BGR)

# Cria a máscara da região de pele
cr_min, cr_max = 133, 173
cb_min, cb_max = 77, 127
mask_skin = np.zeros((size, size), dtype=np.uint8)
mask_skin[cr_min:cr_max+1, cb_min:cb_max+1] = 255  # Cr = linha, Cb = coluna

# Contornos para sobreposição
contours, _ = cv2.findContours(mask_skin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
img_overlay = img_bgr.copy()
cv2.drawContours(img_overlay, contours, -1, (0, 255, 255), 2)  # contorno ciano

# Mostra com eixos centrados (como no gráfico B da sua imagem)
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(img_overlay, cv2.COLOR_BGR2RGB), extent=[0, 1, 0, 1], origin='lower')
plt.title("Espaço Cb × Cr com Faixa de Pele")
plt.xlabel("Cb (azul)")
plt.ylabel("Cr (vermelho)")
plt.grid(True)
plt.show()



In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Carregar imagem
img = cv2.imread('pessoas.jpg')
img = cv2.resize(img, (640, 480))  # Ajusta tamanho
img_blur = cv2.GaussianBlur(img, (5, 5), 0)

# Converter para YCrCb
img_ycrcb = cv2.cvtColor(img_blur, cv2.COLOR_BGR2YCrCb)

# Definir faixa de cor de pele (valores típicos em YCrCb)
lower = np.array([0, 133, 77], dtype=np.uint8)
upper = np.array([255, 173, 127], dtype=np.uint8)

# Criar máscara
skin_mask = cv2.inRange(img_ycrcb, lower, upper)

# TAREFA: Operações morfológicas para remover ruído que gera sobreposição das faces na mascara


# Encontrar contornos na máscara
contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Desenhar contornos com área acima de um limite
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 1000:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        if 0.6 < aspect_ratio < 1.6:  # Aproximação de rosto
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Mostrar resultado
cv2_imshow(img)
cv2_imshow(skin_mask)
